<center><h1>ResNet: MVTEC </h1></center>

## Imports

In [1]:
from __future__ import division,print_function

%matplotlib inline
%load_ext autoreload
%autoreload 2

import sys
from tqdm import tqdm_notebook as tqdm

import random
import matplotlib.pyplot as plt
import math

import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
from torch.autograd import Variable, grad
from torchvision import datasets, transforms as trn
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader

import calculate_log as callog
out_target='color'


# import warnings
# warnings.filterwarnings('ignore')

In [2]:
torch.cuda.set_device(0) #Select the GPU

## Model definition

In [3]:
from PIL import Image

class SaigeDataset3(torch.utils.data.Dataset):
    def __init__(self, data_root, split_root, dataset, split, transform, targets):
        """
            data_root(str) : Root directory of datasets (e.g. "/home/sr2/HDD2/Openset/")
            split_root(str) : Root directroy of split file (e.g. "/home/sr2/Hyeokjun/OOD-saige/datasets/data_split/")
            dataset(str) : dataset name
            split(str) : ['train', 'valid', 'test']
            transform(torchvision transform) : image transform
            targets(list of str) : using targets
        """
        self.data_root = data_root
        self.split_root = split_root
        self.dataset = dataset
        self.transform = transform
        self.targets = targets
        self.data_list = []
        f = open(os.path.join(split_root, split + ".txt"), "r")
        while True:
            line = f.readline()
            if not line: break
            [target, _] = line[:-1].split("/")
            # Transform target
            if target in targets:
                target = targets.index(target)
                self.data_list.append((target, line[:-1]))
                
    def __getitem__(self, idx):
        if isinstance(idx, int):
            (target, fpath) = self.data_list[idx]
            img = Image.open(os.path.join(self.data_root, fpath))
            rgb= img.convert("RGB")
            img = self.transform(rgb)
            return img, target

        else:
            idx=range(100000)[idx]
            if type(idx)==int : idx = [idx]
            for count,x in enumerate(idx):
                try: (target, fpath) = self.data_list[x]
                except: break
                img=Image.open(os.path.join(self.data_root,fpath))
                rgb=img.convert("RGB")
                img= self.transform(rgb)
                img=torch.unsqueeze(img,dim=0)
                if count==0:
                    total_img=[[img,torch.tensor([target])]]
                else:
                    total_img.append([img,torch.tensor([target])])
            return total_img

    
    def data(self):
        return img.data
       
    def __len__(self):
        return len(self.data_list)

class SaigeDataset(torch.utils.data.Dataset):
    def __init__(self, data_root, split_root, dataset, split, transform, targets):
        """
            data_root(str) : Root directory of datasets (e.g. "/home/sr2/HDD2/Openset/")
            split_root(str) : Root directroy of split file (e.g. "/home/sr2/Hyeokjun/OOD-saige/datasets/data_split/")
            dataset(str) : dataset name
            split(str) : ['train', 'valid', 'test']
            transform(torchvision transform) : image transform
            targets(list of str) : using targets
        """
        self.data_root = data_root
        self.split_root = split_root
        self.dataset = dataset
        self.transform = transform
        self.targets = targets
        
        self.data_list = []
        f = open(os.path.join(split_root, split + ".txt"), "r")
        
        while True:
            line = f.readline()
            if not line: break
            [target, _] = line[:-1].split("/")
            # Transform target
            if target in targets:
                target = targets.index(target)
                self.data_list.append((target, line[:-1]))
                
            
    def __getitem__(self, idx):
        if isinstance(idx, int):
            (target, fpath) = self.data_list[idx]
            img = Image.open(os.path.join(self.data_root, fpath))
            img = self.transform(img)
            return img, target
        else:
            idx=range(100000)[idx]
            if type(idx)==int : idx = [idx]
            for count,x in enumerate(idx):
                try: (target, fpath) = self.data_list[x]
                except: break
                img=Image.open(os.path.join(self.data_root,fpath))
                img= self.transform(img)
                img=torch.unsqueeze(img,dim=0)
                if count==0:
                    total_img=[[img,torch.tensor([target])]]
                else:
                    total_img.append([img,torch.tensor([target])])
            return total_img
       
    
    def __len__(self):
        return len(self.data_list)

def getDataLoader(ds_cfg, dl_cfg, split, num_samples=10000):
    if split == 'train':
        train = True
        transform = ds_cfg['train_transform']
    else:
        train = False
        transform = ds_cfg['valid_transform']
        
    if 'split' in ds_cfg.keys() and ds_cfg['split'] == 'train':
           split = 'train'
    elif 'split' in ds_cfg.keys() and ds_cfg['split'] == 'valid':
            split = 'valid'
    elif 'split' in ds_cfg.keys() and ds_cfg['split'] == 'test':
            split = 'test'
    else:
            pass
    if ds_cfg['dataset'] in ['SDI/34Ah','SDI/37Ah','SDI/60Ah']:
        dataset = SaigeDataset(data_root=ds_cfg['data_root'],
                                            split_root=ds_cfg['split_root'],
                                            dataset=ds_cfg['dataset'],
                                            split=split,
                                            transform=transform,
                                            targets=ds_cfg['targets'])
        number= dataset.__len__()
        loader = DataLoader(dataset,
                            batch_size=ds_cfg['batch_size'], shuffle=train,
                            num_workers=dl_cfg['num_workers'], pin_memory=dl_cfg['pin_memory'])
        print('Dataset {} ready.'.format(ds_cfg['dataset']))
    elif 'mvtec' in ds_cfg['dataset']:
        dataset = SaigeDataset(data_root=ds_cfg['data_root'],
                                            split_root=ds_cfg['split_root'],
                                            dataset=ds_cfg['dataset'],
                                            split=split,
                                            transform=transform,
                                            targets=ds_cfg['targets'])
        number= dataset.__len__()
        loader = DataLoader(dataset,
                            batch_size=ds_cfg['batch_size'], shuffle=train,
                            num_workers=dl_cfg['num_workers'], pin_memory=dl_cfg['pin_memory'])
        print('Dataset {} ready.'.format(ds_cfg['dataset']))

    else :
        dataset = SaigeDataset3(data_root=ds_cfg['data_root'],
                                            split_root=ds_cfg['split_root'],
                                            dataset=ds_cfg['dataset'],
                                            split=split,
                                            transform=transform,
                                            targets=ds_cfg['targets'])
        number= dataset.__len__()
        loader = DataLoader(dataset,
                            batch_size=ds_cfg['batch_size'], shuffle=train,
                            num_workers=dl_cfg['num_workers'], pin_memory=dl_cfg['pin_memory'])
        print('Dataset {} ready.'.format(ds_cfg['dataset']))

    return loader

print('Done')

    

Done


In [4]:
def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )
    
    def forward(self, x):
        t = self.conv1(x)
        out = F.relu(self.bn1(t))
        torch_model.record(t)
        torch_model.record(out)
        t = self.conv2(out)
        out = self.bn2(self.conv2(out))
        torch_model.record(t)
        torch_model.record(out)
        t = self.shortcut(x)
        out += t
        torch_model.record(t)
        out = F.relu(out)
        torch_model.record(out)
        
        return out#, out_list


class ResNet_128(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet_128, self).__init__()
        self.in_planes = 64

        self.conv1 = conv3x3(3,64)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool=nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.avgpool=nn.AdaptiveAvgPool2d(output_size=(1,1))
        self.linear = nn.Linear(512*block.expansion, num_classes)
        
        self.collecting = False
    
    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.maxpool(self.relu(self.bn1(self.conv1(x))))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = out.view(out.size(0), -1)
        y = self.linear(out)
        return y
    
    def record(self, t):
        if self.collecting:
            self.gram_feats.append(t)
    
    def gram_feature_list(self,x):
        self.collecting = True
        self.gram_feats = []
        self.forward(x)
        self.collecting = False
        temp = self.gram_feats
        self.gram_feats = []
        return temp
    
    def load(self, path=os.path.join('pre_trained','mvtec_leather_out_'+str(out_target),'ckpt','checkpoint_epoch_200.pyth')):
        tm = torch.load(path,map_location="cpu")        
        self.load_state_dict(tm['model_state'])
    
    def get_min_max(self, data, power):
        mins = []
        maxs = []
        which_layer=6
        batch_size=8
        for i in range(0,len(data),batch_size):
            batch = data[i:i+batch_size].cuda()
            feat_list = self.gram_feature_list(batch)
            for L1,feat_L in enumerate(feat_list):
                if L1%which_layer==0:
                    L=L1//which_layer
                    if L==len(mins):
                        mins.append([None]*len(power))
                        maxs.append([None]*len(power))

                    for p,P in enumerate(power):
                        g_p = G_p(feat_L,P)

                        current_min = g_p.min(dim=0,keepdim=True)[0]
                        current_max = g_p.max(dim=0,keepdim=True)[0]

                        if mins[L][p] is None:
                            mins[L][p] = current_min
                            maxs[L][p] = current_max
                        else:
                            mins[L][p] = torch.min(current_min,mins[L][p])
                            maxs[L][p] = torch.max(current_max,maxs[L][p])
        
        return mins,maxs


    def get_min_max_real(self, data, power):
        mins = []
        maxs = []
        which_layer=6
        batch_size=16
        for i in range(0,len(data),batch_size):
            batch = data[i:i+batch_size].cuda()
            feat_list = self.gram_feature_list(batch)
            for L1,feat_L in enumerate(feat_list):
                if L1%which_layer==0:
                    L=L1//which_layer
                    if L==len(mins):
                        mins.append([None]*len(power))
                        maxs.append([None]*len(power))

                    for p,P in enumerate(power):
                        g_p = G_p_entire(feat_L,P)

                        current_max = g_p.max(dim=0,keepdim=True)[0]
                        current_min = g_p.min(dim=0,keepdim=True)[0]

                        if mins[L][p] is None:
                            mins[L][p] = current_min
                            maxs[L][p] = current_max
                        else:
                            mins[L][p] = torch.min(current_min,mins[L][p])
                            maxs[L][p] = torch.max(current_max,maxs[L][p])
        
        return mins,maxs

    def get_deviations(self,data,power,mins,maxs):
        deviations = []
        which_layer=6
        batch_size=8
        for i in range(0,len(data),batch_size):            
            batch = data[i:i+batch_size].cuda()
            feat_list = self.gram_feature_list(batch)
            batch_deviations = []
            for L1,feat_L in enumerate(feat_list):
                if L1%which_layer==0:
                    L=L1//which_layer
                    dev = 0
                    for p,P in enumerate(power):
                        g_p = G_p(feat_L,P)

                        dev +=  (F.relu(mins[L][p]-g_p)/torch.abs(mins[L][p]+10**-6)).sum(dim=1,keepdim=True)
                        dev +=  (F.relu(g_p-maxs[L][p])/torch.abs(maxs[L][p]+10**-6)).sum(dim=1,keepdim=True)
                    batch_deviations.append(dev.cpu().detach().numpy())
            batch_deviations = np.concatenate(batch_deviations,axis=1)
            deviations.append(batch_deviations)
        deviations = np.concatenate(deviations,axis=0)
        
        return deviations

    def get_entire_Gram(self, data, power, mins_real, maxs_real, PRED):  
        # gram_l_p=dict()
        gram_inside=dict()

        which_layer=6
        count=0
        print("Start Generating Gram Matrices")
        batch_size=32
        for i in tqdm(range(0,len(data),batch_size)):
            batch = data[i:i+batch_size].cuda()
            feat_list = self.gram_feature_list(batch)
            for L1,feat_L in enumerate(feat_list):
                if L1%which_layer==0:
                    L=L1//which_layer
                    # gram_l_p[L] = dict()
                    # gram_new[L] = dict()
                    gram_inside[L]=dict()

                    for p,P in enumerate(power):
                        mins=mins_real[PRED][L][p]
                        # mins=mins.view(1,-1)
                        maxs=maxs_real[PRED][L][p]
                        # maxs=maxs.view(1,-1)
                        g_p=G_p_entire(feat_L,P)
                        # g_p=g_p.view(128,-1)

                        x=torch.zeros(128,g_p.size(1),g_p.size(2))
                        for k in tqdm(range(g_p.size(0)),leave=False):
                            for i in tqdm(range(g_p.size(1)),leave=False):
                                for j in tqdm(range(g_p.size(2)),leave=False):
                                    time.sleep(0.1)
                                    if g_p[k][j][i]>mins_real[PRED][L][p][0][j][i] and g_p[k][j][i]<maxs_real[PRED][L][p][0][j][i]:
                                        x[k][j][i]=1
                        # x=x.view(128,mins_real[PRED][L][p].size(2),-1)
                        if count==0:
                            gram_inside[L][p]=x
                            count=1
                        else :
                            gram_inside[L][p]=torch.cat((gram_inside[L][p],x),dim=0)
                        

        return gram_inside


torch_model = ResNet_128(BasicBlock, [3,4,6,3], num_classes=5)
torch_model.load()
torch_model.cuda()
torch_model.params = list(torch_model.parameters())
torch_model.eval()
print("Done")  

Done


## Datasets

<b>In-distribution Datasets</b>

In [6]:

cfg = dict()

cfg['in_dataset']=dict()
cfg['in_dataset']['dataset']='mvtec_leather'
cfg['in_dataset']['batch_size']=4
x=['color','cut','fold','glue','good_from_train','poke']
x.remove(out_target)
cfg['in_dataset']['targets']=x
cfg['in_dataset']['train_transform']=trn.Compose([trn.RandomHorizontalFlip(),trn.Resize(224),trn.ToTensor()])
cfg['in_dataset']['valid_transform']=trn.Compose([trn.Resize(224),trn.ToTensor()])
# cfg['in_dataset']['data_root']='/HDD0/Openset/Severstal/'
# cfg['in_dataset']['split_root']='/HDD0/Openset/data_split/Severstal/'
cfg['in_dataset']['data_root']='/HDD0/Saige_Database/ETC/mvtec_leather/images'
cfg['in_dataset']['split_root']='/HDD0/Openset/data_split/mvtec_leather/'

cfg['in_dataset']['num_classes']=len(cfg['in_dataset']['targets'])

cfg['dataloader'] = dict()
cfg['dataloader']['num_workers'] = 8
cfg['dataloader']['pin_memory'] = True

train_loader = getDataLoader(ds_cfg=cfg['in_dataset'],
                                dl_cfg=cfg['dataloader'],
                                split="train")
test_loader = getDataLoader(ds_cfg=cfg['in_dataset'],
                                dl_cfg=cfg['dataloader'],
                                split="valid")
# train_loader = torch.utils.data.DataLoader(
#     datasets.SVHN('data', split="train", download=True,
#                    transform=transform_train),
#     batch_size=batch_size, shuffle=True)
# test_loader = torch.utils.data.DataLoader(
#     datasets.SVHN('data', split="test", download=True, transform=transform_test),
#     batch_size=batch_size)


Dataset mvtec_leather ready.
Dataset mvtec_leather ready.


In [7]:
def getDataset(ds_cfg, dl_cfg, split):
    if split == 'train':
        train = True
        transform = ds_cfg['train_transform']
    else:
        train = False
        transform = ds_cfg['valid_transform']
        
    if 'split' in ds_cfg.keys() and ds_cfg['split'] == 'train':
           split = 'train'
    elif 'split' in ds_cfg.keys() and ds_cfg['split'] == 'valid':
            split = 'valid'
    elif 'split' in ds_cfg.keys() and ds_cfg['split'] == 'test':
            split = 'test'
    else:
            pass
    if ds_cfg['dataset'] in ['SDI/34Ah','SDI/37Ah','SDI/60Ah','mvtec_leather']:
        dataset = SaigeDataset(data_root=ds_cfg['data_root'],
                                            split_root=ds_cfg['split_root'],
                                            dataset=ds_cfg['dataset'],
                                            split=split,
                                            transform=transform,
                                            targets=ds_cfg['targets'])
        number= dataset.__len__()
        loader = DataLoader(dataset,
                            batch_size=ds_cfg['batch_size'], shuffle=train,
                            num_workers=dl_cfg['num_workers'], pin_memory=dl_cfg['pin_memory'])
        print('Dataset {} ready.'.format(ds_cfg['dataset']))
    else :
        dataset = SaigeDataset3(data_root=ds_cfg['data_root'],
                                            split_root=ds_cfg['split_root'],
                                            dataset=ds_cfg['dataset'],
                                            split=split,
                                            transform=transform,
                                            targets=ds_cfg['targets'])
        number= dataset.__len__()
        loader = DataLoader(dataset,
                            batch_size=ds_cfg['batch_size'], shuffle=train,
                            num_workers=dl_cfg['num_workers'], pin_memory=dl_cfg['pin_memory'])
        print('Dataset {} ready.'.format(ds_cfg['dataset']))

    return dataset


data_train = getDataset(ds_cfg=cfg['in_dataset'],
                                dl_cfg=cfg['dataloader'],
                                split="train")

# data_train = list(list(torch.utils.data.DataLoader(
#     datasets.SVHN('data', split="train", download=True,transform=cfg['in_dataset']['dataset']),batch_size=1, shuffle=True)))

Dataset mvtec_leather ready.


In [8]:
data = getDataset(ds_cfg=cfg['in_dataset'],
                                dl_cfg=cfg['dataloader'],
                                split="valid")

Dataset mvtec_leather ready.


In [9]:
torch_model.eval()
correct = 0
total = 0
for x,y in test_loader:
    x = x.cuda()
    y = y.numpy()
    correct += (y==np.argmax(torch_model(x).detach().cpu().numpy(),axis=1)).sum()
    total += y.shape[0]
print("Accuracy: ",correct/total)


Accuracy:  0.9230769230769231


<b>Out-of-distribution Datasets</b>

## Code for Detecting OODs

<b> Extract predictions for train and test data </b>

In [10]:
# from ipywidgets import IntProgress

train_preds = []
train_confs = []
train_logits = []
batch_size=16
for idx in tqdm(range(0,len(data_train),batch_size)):

    # batch = torch.squeeze(batch, dim=1).cuda()
    batch = torch.squeeze(torch.stack([x[0] for x in data_train[idx:idx+batch_size]]),dim=1).cuda()
    
    logits = torch_model(batch)
    confs = F.softmax(logits,dim=1).cpu().detach().numpy()
    preds = np.argmax(confs,axis=1)
    logits = (logits.cpu().detach().numpy())

    train_confs.extend(np.max(confs,axis=1))    
    train_preds.extend(preds)
    train_logits.extend(logits)
print("Done")

test_preds = []
test_confs = []
test_logits = []

for idx in tqdm(range(0,len(data),batch_size)):
    batch = torch.squeeze(torch.stack([x[0] for x in data[idx:idx+batch_size]]),dim=1).cuda()
    
    logits = torch_model(batch)
    confs = F.softmax(logits,dim=1).cpu().detach().numpy()
    preds = np.argmax(confs,axis=1)
    logits = (logits.cpu().detach().numpy())

    test_confs.extend(np.max(confs,axis=1))    
    test_preds.extend(preds)
    test_logits.extend(logits)
print("Done")


Done



Done


<b> Code for detecting OODs by identifying anomalies in correlations </b>

In [11]:
import calculate_log as callog

def detect(all_test_deviations,all_ood_deviations, verbose=True, normalize=True):
    average_results = {}
    for i in range(1,11):
        random.seed(i)
        
        validation_indices = random.sample(range(len(all_test_deviations)),int(0.1*len(all_test_deviations)))
        test_indices = sorted(list(set(range(len(all_test_deviations)))-set(validation_indices)))

        validation = all_test_deviations[validation_indices]
        test_deviations = all_test_deviations[test_indices]

        t95 = validation.mean(axis=0)+10**-7
        if not normalize:
            t95 = np.ones_like(t95)
        test_deviations = (test_deviations/t95[np.newaxis,:]).sum(axis=1)
        ood_deviations = (all_ood_deviations/t95[np.newaxis,:]).sum(axis=1)
        
        results = callog.compute_metric(-test_deviations,-ood_deviations)
        for m in results:
            average_results[m] = average_results.get(m,0)+results[m]
    
    for m in average_results:
        average_results[m] /= i
    if verbose:
        callog.print_results(average_results)
    return average_results

def cpu(ob):
    for i in range(len(ob)):
        for j in range(len(ob[i])):
            ob[i][j] = ob[i][j].cpu()
    return ob

def cuda(ob):
    for i in range(len(ob)):
        for j in range(len(ob[i])):
            ob[i][j] = ob[i][j].cuda()
    return ob

class Detector:
    def __init__(self):
        self.all_test_deviations = None
        self.mins = {}
        self.maxs = {}
        self.mins_real = {}
        self.maxs_real = {}
        self.gram_inside_ind = dict()
        self.gram_inside_ood = dict()
        self.classes = range(5)
    
    def compute_minmaxs(self,data_train,POWERS=[10]):
        print("Start")
        for PRED in tqdm(self.classes):
            print('current prediction: {}'.format(PRED))
            train_indices = np.where(np.array(train_preds)==PRED)[0]
            # if train_indices==0: break
            print(len(train_indices))
            train_PRED = torch.squeeze(torch.stack([data_train[i][0][0] for i in train_indices]),dim=1)
            mins,maxs = torch_model.get_min_max(train_PRED,power=POWERS)
            # mins_real,maxs_real = torch_model.get_min_max_real(train_PRED,power=POWERS)
            # gram_l_p = torch_model.get_entire_Gram(train_PRED,power=POWERS)
            # self.grams[PRED]=gram_l_p
            self.mins[PRED] = cpu(mins)
            self.maxs[PRED] = cpu(maxs)
            # self.mins_real[PRED]=cpu(mins_real)
            # self.maxs_real[PRED]=cpu(maxs_real)
            torch.cuda.empty_cache()
    
    def compute_test_deviations(self,data,POWERS=[10]):
        all_test_deviations = None
        test_classes = []
        for PRED in tqdm(self.classes):
            test_indices = np.where(np.array(test_preds)==PRED)[0]
            # if len(test_indices)==0:
            if len(test_indices)==0: continue
            #     print('Passing')
            #     pass
            # print(type(data[1][0]))
            # for i in test_indices:
            #     print(data[i][0])
            x=torch.stack([data[i][0][0] for i in test_indices])
            test_PRED = torch.squeeze(x,dim=1)
            # gram_inside = torch_model.get_entire_Gram(test_PRED,power=POWERS, mins_real=self.mins_real, maxs_real=self.maxs_real, PRED=PRED)
            # self.gram_inside_ind[PRED]=gram_inside
            test_confs_PRED = np.array([test_confs[i] for i in test_indices])
            
            test_classes.extend([PRED]*len(test_indices))
            
            mins = cuda(self.mins[PRED])
            maxs = cuda(self.maxs[PRED])
            test_deviations = torch_model.get_deviations(test_PRED,power=POWERS,mins=mins,maxs=maxs)/test_confs_PRED[:,np.newaxis]
            cpu(mins)
            cpu(maxs)
            if all_test_deviations is None:
                all_test_deviations = test_deviations
            else:
                all_test_deviations = np.concatenate([all_test_deviations,test_deviations],axis=0)
            torch.cuda.empty_cache()
        self.all_test_deviations = all_test_deviations
        
        self.test_classes = np.array(test_classes)
    
    def compute_ood_deviations(self,ood,POWERS=[10]):
        ood_preds = []
        ood_confs = []
        batch_size=16
        for idx in range(0,len(ood),batch_size):
            batch = torch.squeeze(torch.stack([x[0] for x in ood[idx:idx+batch_size]]),dim=1).cuda()
            logits = torch_model(batch)
            confs = F.softmax(logits,dim=1).cpu().detach().numpy()
            preds = np.argmax(confs,axis=1)
            
            ood_confs.extend(np.max(confs,axis=1))
            ood_preds.extend(preds)  
            torch.cuda.empty_cache()
        print("Done")
        
        ood_classes = []
        all_ood_deviations = None
        for PRED in tqdm(self.classes):
            ood_indices = np.where(np.array(ood_preds)==PRED)[0]
            if len(ood_indices)==0:
                continue
            ood_classes.extend([PRED]*len(ood_indices))
            
            ood_PRED = torch.squeeze(torch.stack([ood[i][0][0] for i in ood_indices]),dim=1)
            # gram_entire_values,gram_entire_indexes = torch_model.get_entire_Gram(ood_PRED,power=POWERS)
            # self.grams_values[PRED]=gram_entire_values
            # self.grams_indexes[PRED]=gram_entire_indexes

            ood_confs_PRED =  np.array([ood_confs[i] for i in ood_indices])
            mins = cuda(self.mins[PRED])
            maxs = cuda(self.maxs[PRED])
            ood_deviations = torch_model.get_deviations(ood_PRED,power=POWERS,mins=mins,maxs=maxs)/ood_confs_PRED[:,np.newaxis]
            cpu(self.mins[PRED])
            cpu(self.maxs[PRED])            
            if all_ood_deviations is None:
                all_ood_deviations = ood_deviations
            else:
                all_ood_deviations = np.concatenate([all_ood_deviations,ood_deviations],axis=0)
            torch.cuda.empty_cache()
            
        self.ood_classes = np.array(ood_classes)
        
        average_results = detect(self.all_test_deviations,all_ood_deviations)
        return average_results, self.all_test_deviations, all_ood_deviations


    # def compute_gram_matrices(self,ood,POWERS=[10]):        
    #     for idx in range(0,len(ood),128):
    #         batch = torch.squeeze(torch.stack([x[0] for x in ood[idx:idx+128]]),dim=1).cuda()
    #         logits = torch_model(batch)
    #         confs = F.softmax(logits,dim=1).cpu().detach().numpy()
    #         preds = np.argmax(confs,axis=1)
            
    #         ood_confs.extend(np.max(confs,axis=1))
    #         ood_preds.extend(preds)  
    #         torch.cuda.empty_cache()
    #     print("Done")
        
    #     ood_classes = []
    #     all_ood_deviations = None
    #     for PRED in tqdm(self.classes):
    #         ood_indices = np.where(np.array(ood_preds)==PRED)[0]
    #         if len(ood_indices)==0:
    #             continue
    #         ood_classes.extend([PRED]*len(ood_indices))
            
    #         ood_PRED = torch.squeeze(torch.stack([ood[i][0] for i in ood_indices]),dim=1)
    #         # gram_l_p = torch_model.get_entire_Gram(ood_PRED,power=POWERS)
    #         # self.grams[PRED]=gram_l_p
    #         ood_confs_PRED =  np.array([ood_confs[i] for i in ood_indices])
    #         mins = cuda(self.mins[PRED])
    #         maxs = cuda(self.maxs[PRED])
    #         ood_deviations = torch_model.get_deviations(ood_PRED,power=POWERS,mins=mins,maxs=maxs)/ood_confs_PRED[:,np.newaxis]
    #         cpu(self.mins[PRED])
    #         cpu(self.maxs[PRED])            
    #         if all_ood_deviations is None:
    #             all_ood_deviations = ood_deviations
    #         else:
    #             all_ood_deviations = np.concatenate([all_ood_deviations,ood_deviations],axis=0)
    #         torch.cuda.empty_cache()
            
    #     self.ood_classes = np.array(ood_classes)
        
    #     average_results = detect(self.all_test_deviations,all_ood_deviations)
    #     return average_results, self.all_test_deviations, all_ood_deviations


<center><h1> Results </h1></center>

In [12]:
import time

def G_p(ob, p):
    temp = ob.detach()
    
    temp = temp**p
    temp = temp.reshape(temp.shape[0],temp.shape[1],-1)
    temp = ((torch.matmul(temp,temp.transpose(dim0=2,dim1=1)))).sum(dim=2) 
    temp = (temp.sign()*torch.abs(temp)**(1/p)).reshape(temp.shape[0],-1)
    
    return temp

def G_p_entire(ob, p):
    temp = ob.detach()
    temp = temp**p
    temp = temp.reshape(temp.shape[0],temp.shape[1],-1)
    temp = ((torch.matmul(temp,temp.transpose(dim0=2,dim1=1))))
    # temp=temp.sum(dim=0)
    temp = (temp.sign()*torch.abs(temp)**(1/p)).reshape(temp.shape[0],temp.shape[1],-1)
    return temp

detector = Detector()
detector.compute_minmaxs(data_train,POWERS=range(1,10))


Start


current prediction: 0
15
current prediction: 1
13
current prediction: 2
15
current prediction: 3
51
current prediction: 4
14



In [13]:

detector.compute_test_deviations(data,POWERS=range(1,10))



<b> OOD Detection Performance for NG Classes </b>

In [14]:
cfg['out_dataset']=dict()
cfg['out_dataset']['dataset']='mvtec_leather'
cfg['out_dataset']['batch_size']=4
# x=['0','1','2','3','4','5','6','7','8','9']
# x.remove(str(1))
cfg['out_dataset']['targets']=[out_target]#[str(2)]
cfg['out_dataset']['train_transform']=trn.Compose([trn.RandomHorizontalFlip(),trn.Resize(224),trn.ToTensor()])
cfg['out_dataset']['valid_transform']=trn.Compose([trn.Resize(224),trn.ToTensor()])
cfg['out_dataset']['data_root']='/HDD0/Saige_Database/ETC/mvtec_leather/images'
cfg['out_dataset']['split_root']='/HDD0/Openset/data_split/mvtec_leather'
cfg['out_dataset']['num_classes']=len(cfg['in_dataset']['targets'])

cfg['dataloader'] = dict()
cfg['dataloader']['num_workers'] = 8
cfg['dataloader']['pin_memory'] = True

out_dataset = getDataset(ds_cfg=cfg['out_dataset'],
                                dl_cfg=cfg['dataloader'],
                                split="valid")
print("\n\n*************")
print("Real Out Target :{}, target {}-{}".format(cfg['out_dataset']['dataset'],cfg['out_dataset']['targets'][0]))
c10_results = detector.compute_ood_deviations(out_dataset,POWERS=range(1,10))
print("*************\n\n")


Dataset mvtec_leather ready.


*************
Real Out Target :mvtec_leather, target color-NG
Done



 TNR    AUROC  DTACC  AUIN   AUOUT 
  0.000  0.000 50.000 67.484  7.487
*************




<b> OOD Detection Performance for left out OK Class </b>